# Project Report
Group 60: Daniyal, Ruth, Thomas, Paul

## Title

### Introduction

### Methods & Results
Our analysis makes use of the `repr`, `tidyverse`, and `tidymodels` packages in R.

In [1]:
# Setup
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)
options(repr.plot.width = 8, repr.plot.height = 7)
set.seed(9248)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [17]:
# Obtaining and cleaning data
heart_raw <- read_delim("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/new.data",
                        col_names = FALSE, na = "-9") |>
    separate(X7, c("X7", "X8"), sep = " ", convert = TRUE) |>
    na_if(-9)
heart_raw
row_count <- nrow(heart_raw) / 12
row_indexes <- tibble(id = rep(1:row_count, each = 12), Y = rep(1:12, times = row_count))
heart_indexed <- bind_cols(row_indexes, heart_raw)
heart_indexed
heart_wide <- heart_indexed |>
    pivot_wider(names_from = Y, values_from = X1:X8, names_vary = "slowest") |>
    select(!c(id, X8_1, X4_12:X8_12)) |>
    type_convert() |>
    select_if(\(x) sum(is.na(x)) != row_count)
colnames(heart_wide) <- 1:ncol(heart_wide)
heart_wide
# next: give columns proper names and types, figure out why there are 83 instead of 76

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 18492 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: " "
chr (2): X3, X7
dbl (5): X1, X2, X4, X5, X6

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 2303 rows [12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180, 192, 204, 216, 228, 240, ...].”


X1,X2,X3,X4,X5,X6,X7,X8
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,15943882,63,1,NA,NA,NA,NA
-27,1,145,1,233,NA,50,20
1,0,1,2,2,3,1981,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1,1,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,Takacs,NA,NA,NA,NA,NA


id,Y,X1,X2,X3,X4,X5,X6,X7,X8
<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,15943882,63,1,NA,NA,NA,NA
1,2,-27,1,145,1,233,NA,50,20
1,3,1,0,1,2,2,3,1981,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1541,10,1,1,NA,NA,NA,NA,NA,NA
1541,11,NA,NA,NA,NA,NA,NA,NA,NA
1541,12,NA,NA,Takacs,NA,NA,NA,NA,NA



── Column specification ────────────────────────────────────────────────────────
cols(
  X3_1 = col_double(),
  X3_2 = col_double(),
  X3_3 = col_double(),
  X3_4 = col_double(),
  X3_5 = col_double(),
  X3_6 = col_double(),
  X3_7 = col_double(),
  X3_8 = col_double(),
  X3_9 = col_double(),
  X3_10 = col_double(),
  X3_11 = col_double(),
  X3_12 = col_character()
)



1,2,3,4,5,6,7,8,9,10,⋯,74,75,76,77,78,79,80,81,82,83
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,15943882,63,1,NA,NA,NA,-27,1,145,⋯,NA,NA,NA,NA,NA,0,0,0,0,name
2,15964847,67,1,NA,NA,NA,-27,4,160,⋯,NA,NA,NA,NA,NA,0,0,0,0,name
3,15952199,67,1,NA,NA,NA,-27,4,120,⋯,NA,NA,NA,NA,NA,0,0,0,0,name
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1668,13203,57,1,1,NA,NA,NA,4,130,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Torok
1669,14205,47,1,1,NA,NA,NA,4,100,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Veszelsz
1670,15403,36,1,1,NA,NA,NA,3,120,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Takacs


### Discussion

### References